In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
def unzip(src_dir,new_name = None):
	# extract to current directory
	dirpath = '.'
	try:
		if src_dir.endswith('.zip'):
			print('unzipping ' + src_dir)
			with zipfile.ZipFile(src_dir) as zf:
				zip_dir = zf.namelist()[0]
				zf.extractall(dirpath)
		elif src_dir.endswith('.tgz') or src_dir.endswith('tar.gz'):
			print('unzipping ' + src_dir)
			tar = tarfile.open(src_dir)
			tar.extractall()
			tar.close()
		# os.remove(save_path)
		if new_name is None:
			pass
		else:
			os.rename('jpg', os.path.join(dirpath, new_name))
	except:
		raise('wrong format')

In [4]:
import tarfile
import os
image_dir="/kaggle/input/flowers-text-to-image/102flowers.tgz"
unzip(image_dir,'102flowers')


unzipping /kaggle/input/flowers-text-to-image/102flowers.tgz


In [11]:
def load_folder_list(path=""):
    """Return a folder list in a folder by given a folder path.
    Parameters
    ----------
    path : a string or None
        A folder path.
    """
    return [os.path.join(path,o) for o in os.listdir(path) if os.path.isdir(os.path.join(path,o))]

In [5]:
import sys
from contextlib import contextmanager
@contextmanager
def suppress_stdout():
    """Temporarily disable console output.
    Examples
    ---------
    >>> print("You can see me")
    >>> with tl.ops.suppress_stdout():
    >>>     print("You can't see me")
    >>> print("You can see me")
    References
    -----------
    - `stackoverflow <http://stackoverflow.com/questions/2125702/how-to-suppress-console-output-in-python>`_
    """
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout


In [8]:
pip install tensorlayer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.2/381.2 kB 1.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install scipy==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 31.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
woodwork 0.16.4 requires scipy>=1.4.0, but you have scipy 1.2.2 which is incompatible.
tpot 0.11.7 requires scipy>=1.3.1, but you have scipy 1.2.2 which is incompatible.
stumpy 1.11.1 requires scipy>=1.5, but you have scipy 1.2.2 which is incompatible.
statsmodels 0.13.2 requires scipy>=1.3, but you have scipy 1.2.2 which is incompatible.
sklearn-pandas 2.2.0 requires scipy>=1.5.1, but you have scipy 1.2.2 which is incompatible.
scikit-image 0.19.3 requires scipy>=1.4.1, but you have scipy 1.2.2 which is incompatible.
quantecon 0.6.0 requires scipy>=1.5.0, but you have scipy 1.2.

In [14]:
help('scipy.misc.imresize')

No Python documentation found for 'scipy.misc.imresize'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.



In [ ]:
import os
import re
import time
import nltk
import re
import string
import imageio
import skimage
import tensorlayer as tl
from utils import *


dataset = '102flowers' #
need_256 = True # set to True for stackGAN



if dataset == '102flowers':
    """
    images.shape = [8000, 64, 64, 3]
    captions_ids = [80000, any]
    """
    cwd = os.getcwd()
    img_dir = os.path.join(cwd, '102flowers')
    caption_dir = os.path.join(cwd, '/kaggle/input/flowers-dataset/text_c10')
    VOC_FIR = cwd + '/vocab.txt'

    ## load captions
    caption_sub_dir = load_folder_list( caption_dir )
    captions_dict = {}
    processed_capts = []
    for sub_dir in caption_sub_dir: # get caption file list
        with suppress_stdout():
            files = tl.files.load_file_list(path=sub_dir, regx='^image_[0-9]+\.txt')
            for i, f in enumerate(files):
                file_dir = os.path.join(sub_dir, f)
                key = int(re.findall('\d+', f)[0])
                t = open(file_dir,'r')
                lines = []
                for line in t:
                    line = preprocess_caption(line)
                    lines.append(line)
                    processed_capts.append(tl.nlp.process_sentence(line, start_word="<S>", end_word="</S>"))
                assert len(lines) == 10, "Every flower image have 10 captions"
                captions_dict[key] = lines
    print(" * %d x %d captions found " % (len(captions_dict), len(lines)))

    ## build vocab
    if not os.path.isfile('vocab.txt'):
        _ = tl.nlp.create_vocab(processed_capts, word_counts_output_file=VOC_FIR, min_word_count=1)
    else:
        print("WARNING: vocab.txt already exists")
    vocab = tl.nlp.Vocabulary(VOC_FIR, start_word="<S>", end_word="</S>", unk_word="<UNK>")

    ## store all captions ids in list
    captions_ids = []
    try: # python3
        tmp = captions_dict.items()
    except: # python3
        tmp = captions_dict.iteritems()
    for key, value in tmp:
        for v in value:
            captions_ids.append( [vocab.word_to_id(word) for word in nltk.tokenize.word_tokenize(v)] + [vocab.end_id])  # add END_ID
            # print(v)              # prominent purple stigma,petals are white inc olor
            # print(captions_ids)   # [[152, 19, 33, 15, 3, 8, 14, 719, 723]]
            # exit()
    captions_ids = np.asarray(captions_ids)
    print(" * tokenized %d captions" % len(captions_ids))

    ## check
    img_capt = captions_dict[1][1]
    print("img_capt: %s" % img_capt)
    print("nltk.tokenize.word_tokenize(img_capt): %s" % nltk.tokenize.word_tokenize(img_capt))
    img_capt_ids = [vocab.word_to_id(word) for word in nltk.tokenize.word_tokenize(img_capt)]#img_capt.split(' ')]
    print("img_capt_ids: %s" % img_capt_ids)
    print("id_to_word: %s" % [vocab.id_to_word(id) for id in img_capt_ids])

    ## load images
    with suppress_stdout():  # get image files list
        imgs_title_list = sorted(tl.files.load_file_list(path=img_dir, regx='^image_[0-9]+\.jpg'))
    print(" * %d images found, start loading and resizing ..." % len(imgs_title_list))
    s = time.time()

    # time.sleep(10)
    # def get_resize_image(name):   # fail
    #         img = scipy.misc.imread( os.path.join(img_dir, name) )
    #         img = tl.prepro.imresize(img, size=[64, 64])    # (64, 64, 3)
    #         img = img.astype(np.float32)
    #         return img
    # images = tl.prepro.threading_data(imgs_title_list, fn=get_resize_image)
    images = []
    images_256 = []
    for name in imgs_title_list:
        # print(name)
        img_raw = imageio.imread( os.path.join(img_dir, name) )
        # (64, 64, 3)
        img=skimage.transform.resize(img_raw,[64,64])
        img = img.astype(np.float32)
        images.append(img)
        if need_256:
            img = skimage.transform.resize(img_raw,[256, 256]) # (256, 256, 3)
            img = img.astype(np.float32)

            images_256.append(img)
    # images = np.array(images)
    # images_256 = np.array(images_256)
    print(" * loading and resizing took %ss" % (time.time()-s))

    n_images = len(captions_dict)
    n_captions = len(captions_ids)
    n_captions_per_image = len(lines) # 10

    print("n_captions: %d n_images: %d n_captions_per_image: %d" % (n_captions, n_images, n_captions_per_image))

    captions_ids_train, captions_ids_test = captions_ids[: 8000*n_captions_per_image], captions_ids[8000*n_captions_per_image :]
    images_train, images_test = images[:8000], images[8000:]
    if need_256:
        images_train_256, images_test_256 = images_256[:8000], images_256[8000:]
    n_images_train = len(images_train)
    n_images_test = len(images_test)
    n_captions_train = len(captions_ids_train)
    n_captions_test = len(captions_ids_test)
    print("n_images_train:%d n_captions_train:%d" % (n_images_train, n_captions_train))
    print("n_images_test:%d  n_captions_test:%d" % (n_images_test, n_captions_test))

    ## check test image
    # idexs = get_random_int(min=0, max=n_captions_test-1, number=64)
    # temp_test_capt = captions_ids_test[idexs]
    # for idx, ids in enumerate(temp_test_capt):
    #     print("%d %s" % (idx, [vocab.id_to_word(id) for id in ids]))
    # temp_test_img = images_train[np.floor(np.asarray(idexs).astype('float')/n_captions_per_image).astype('int')]
    # save_images(temp_test_img, [8, 8], 'temp_test_img.png')
    # exit()

    # ## check the first example
    # tl.visualize.frame(I=images[0], second=5, saveable=True, name='temp', cmap=None)
    # for cap in captions_dict[1]:
    #     print(cap)
    # print(captions_ids[0:10])
    # for ids in captions_ids[0:10]:
    #     print([vocab.id_to_word(id) for id in ids])
    # print_dict(captions_dict)

    # ## generate a random batch
    # batch_size = 64
    # idexs = get_random_int(0, n_captions_test, batch_size)
    # # idexs = [i for i in range(0,100)]
    # print(idexs)
    # b_seqs = captions_ids_test[idexs]
    # b_images = images_test[np.floor(np.asarray(idexs).astype('float')/n_captions_per_image).astype('int')]
    # print("before padding %s" % b_seqs)
    # b_seqs = tl.prepro.pad_sequences(b_seqs, padding='post')
    # print("after padding %s" % b_seqs)
    # # print(input_images.shape)   # (64, 64, 64, 3)
    # for ids in b_seqs:
    #     print([vocab.id_to_word(id) for id in ids])
    # print(np.max(b_images), np.min(b_images), b_images.shape)
    # from utils import *
    # save_images(b_images, [8, 8], 'temp2.png')
    # # tl.visualize.images2d(b_images, second=5, saveable=True, name='temp2')
    # exit()

import pickle
def save_all(targets, file):
    with open(file, 'wb') as f:
        pickle.dump(targets, f)

save_all(vocab, '_vocab.pickle')
save_all((images_train_256, images_train), '_image_train.pickle')
#save_all((images_test_256, images_test), '_image_test.pickle')
save_all((n_captions_train, n_captions_test, n_captions_per_image, n_images_train, n_images_test), '_n.pickle')
save_all((captions_ids_train, captions_ids_test), '_caption.pickle')

 * 8189 x 10 captions found 


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:95: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.


 * tokenized 81890 captions
img_capt: this flower has bright purple  spiky petals  and greenish sepals below them 
nltk.tokenize.word_tokenize(img_capt): ['this', 'flower', 'has', 'bright', 'purple', 'spiky', 'petals', 'and', 'greenish', 'sepals', 'below', 'them']
img_capt_ids: [6, 3, 7, 31, 18, 165, 4, 5, 318, 83, 374, 120]
id_to_word: ['this', 'flower', 'has', 'bright', 'purple', 'spiky', 'petals', 'and', 'greenish', 'sepals', 'below', 'them']
 * 8189 images found, start loading and resizing ...
 * loading and resizing took 659.7158839702606s
n_captions: 81890 n_images: 8189 n_captions_per_image: 10
n_images_train:8000 n_captions_train:80000
n_images_test:189  n_captions_test:1890


In [6]:
pip install utils

Note: you may need to restart the kernel to use updated packages.
